# Process events from the Watson IoT Platform in a Streams Python application

If you have data being sent from an Internet of Things device to the Watson IoT platform, you can perform advanced analytics on such data using Python and the Streaming Analytics service.  This notebook shows a simple Python Streams application that connects to the Watson IoT platform to retrieve data sent there by an Edgent application.


## Table of contents

1. [Prerequisites](#setup)<br>
    1.1 [Option 1: Automatically deploy and configure the services](#setup1)<br>
    1.2 [Option 2: Manual deployment](#setup2)<br>
    1.3 [Run a sample Edgent application to generate data](#setup3)<br>
    1.4 [Install the required version of the streamsx package](#setup4)<br>
2. [Create and submit the Streams application](#streams)<br>
    2.1 [Get your Streaming analytics service credentials](#step21)<br>
    2.2 [Define helper functions](#step22)<br>
    2.3 [Create the Streams application](#step23)<br>
    2.4 [Send a response for each event](#step24)<br>
    2.5 [Submit the `Topology` to the Streaming Analytics service](step25)<br>
3.	[Visualize data](#view)<br>
4.  [Stop the application](#stop)<br>
5. [Summary and next steps](#summary)<br>

<a id="setup"></a>
## 1. Setup

This notebook requires you to have the Streaming Analytics service and the Watson IoT Platform service. You can set up and connect the services manually or use the automatic option if it applies to you. This notebook is a companion to the <a href="https://developer.ibm.com/recipes/tutorials/connect-apache-edgent-to-the-streaming-analytics-service-using-the-watson-iot-platform/" target="_blank">Connect Apache Edgent running on Raspberry Pi to the Streaming Analytics service recipe</a>. It will show you a  Python Streams application that receives IoT device events from the Watson IoT platform. This is a Python version of the Java application demonstrated in that recipe. 
 
Therefore, you must complete the steps in this section to create the following:
- An instance of the Watson IoT platform and the Streaming Analytics service
- An Edgent application or other application that is registered with and sending events to the Watson IoT platform.
- `IotPlatformBluemix` or `IotPlatform`  application running in your Streams instance.

<a id="setup1"></a>
### 1.1 Option 1: Automatically deploy and configure the services

You can use this option if:
- You do not have the services service created in IBM Cloud, or
- You have both services created and would like to use automatic configuration. If so, you need to rename the services to match the names expected by the configuration script before starting the process:
   - Rename the Streaming Analtyics service to *Streaming-Analytics* 
   - Rename the Watson IoT Platform to *Internet-of-Things-Platform*.
   *These names must match exactly as indicated here*.

 1. Click the **Deploy to IBM Cloud** button to deploy the services automatically. 
    [![Deploy To IBM Cloud](https://developer.ibm.com/streamsdev/wp-content/uploads/sites/15/2017/11/deploy-to-ibm-cloud-small.png)](https://bluemix.net/deploy?repository=https://github.com/IBMStreams/streamsx.iot.starterkit.git)
 2. Once the deployment is finished, go to your IBM Cloud dashboard as shown below. Click the newly deployed app under "Cloud Foundry Apps". This will take you to the application page.
    ![app dashboard](https://developer.ibm.com/streamsdev/wp-content/uploads/sites/15/2018/01/app-dashboard.png)

    Set up a username and password for the starter kit by following these instructions:
    - Click **Runtime** > **Environment variables**.
    - Under **User defined**, create a username and password for your starter kit by adding 2 variables called `KIT_OWNER` (username) and `KIT_PASSWORD` (password). Names must be exactly as shown here.  Click **Save**.
    - Click **Visit app URL** to go to the starter kit home page and log in with the username and password you just created.
    These steps are illustrated below.
    ![setusername](https://raw.githubusercontent.com/IBMStreams/streamsx.iot.starterkit/master/img/env.png)

 3. Clicking **Visit App URL** will take you to the home page of your starter kit. Log in using the `KIT_OWNER` and `KIT_PASSWORD` as username and password. Now you can access the needed credentials.
    To finish the automatic setup: 

    1. Go to the home page of the starter kit:
    1. Go to "Tools" and click "Submit IotPlatform Job".
    1. Download the `device.cfg` file: Click **View All Credentials**, then under **Edgent Credentials**, select **Download Device.cfg**. Save this file locally.

<a id="setup2"></a>
### 1.2 Option 2: Manual deployment 

If you have not already done so, create instances of the <a href="https://console.bluemix.net/catalog/services/streaming-analytics" target="_blank">Streaming Analytics service</a> and the <a href="https://console.bluemix.net/catalog/services/internet-of-things-platform" target="_blank">Watson IoT Platform</a> service. 

Next, follow the <a href="https://developer.ibm.com/streamsdev/docs/setup-instructions-connecting-edgent-streams-applications-watson-iot-platform" target="_blank">instructions in this post</a>.
After completing the steps in the above article, you will get:
 - A registered device on the Watson IoT Platform and a `device.cfg` file with the information for the device.
 - The `IotPlatformBluemix` application running in your Streaming Analytics application.

<a id="setup3"></a>
### 1.3 Run a sample Edgent application to generate data 

he `IotpSensors` application in Edgent generates the data that will be processed in this notebook.
To run the application and start generating data:

1. [Download and unpack Edgent](https://edgent.apache.org/docs/downloads)
1. For Edgent 1.1.0 and greater, edit the file $EDGENT/java8/scripts/connectors/iotp/runiotpsensors.sh to uncomment the line:
   `USE_OLD_EVENT_FORMAT=-Dcom.ibm.iotf.enableCustomFormat=false.`
   
1. Run the sample script, using the device.cfg file you created earlier
```
cd $EDGENT/java8/scripts/connectors/iotp/
./runiotpsensors.sh path_to_device_cfg:
```

You should see output like this:
```
"when":"2017-07-11T20:04:31Z","time":1499803471283}
Jul 11, 2017 4:04:31 PM com.ibm.iotf.client.AbstractClient connect
INFO: pool-1-thread-19-IotpSensors: Successfully connected to the IBM Watson IoT Platform
{"name":"B","reading":{"N":4,"MIN":-2.525664438740216,"MAX":-1.7670767522693176,"MEAN":-2.0834865122722173,"STDDEV":0.3294944133737305}}
```
Now you are ready to process those events in Streams.


<a id="setup4"></a>
### 1.4 Install the required version of the streamsx package
This notebook requires version 1.7 or later of the streamsx package.  
Run the cell below to determine which version is installed. 

In [1]:
!pip show streamsx

---
Metadata-Version: 2.0
Name: streamsx
Version: 1.6.0
Summary: IBM Streams Python Support
Home-page: https://github.com/IBMStreams/pypi.streamsx
Author: IBM Streams @ github.com
Author-email: debrunne@us.ibm.com
Installer: pip
License: UNKNOWN
Location: /usr/local/src/conda3_runtime.v22/4.1.1/lib/python3.5/site-packages
Requires: enum34, requests, future, dill
Classifiers:
  Development Status :: 5 - Production/Stable
  License :: OSI Approved :: Apache Software License
  Programming Language :: Python :: 2
  Programming Language :: Python :: 2.7
  Programming Language :: Python :: 3
  Programming Language :: Python :: 3.5


After running the above cell, you will see output like this:
```
---
Metadata-Version: 2.0
Name: streamsx
Version: 1.7.4
....
```
If `Version` is not at or greater than 1.7, uncomment and run the following cell.

In [ ]:
#Comment out the line below if needed
#!pip install --upgrade streamsx

<a id="streams"></a>
## 2. Create and submit the Streams application

To create and submit your Streams application you must perform these steps:

1. [Get your Streaming analytics service credentials](#step21)<br>
1. [Define helper functions](#step22)<br>
1. [Create the Streams application](#step23)<br>
1. [Send a response for each event](#step24)<br>
1. [Submit the `Topology` to the Streaming Analytics service](step25)<br>

<a id="step21"></a>
### 2.1 Get your Streaming analytics service credentials
Before you create an app with this notebook, you must first provide the information that your streaming app needs to access the service. You can find this information on the Streaming Analytics service dashboard. It includes the service name, the service credentials, and the connection URL for the service. 
In the next cell you have to enter your service credentials. To copy your service credentials, open the Streaming Analytics service dashboard click **Service Credentials**, then **View Credentials**, and finally click the Copy icon.

<img src='https://github.com/orzade/streamsx-notebooks/blob/master/copyservicecredentials.png?raw=true' alt="Copy your service credentials" title="Streaming Analytics catalog - Copy your service credentials"></img>

Then paste those credentials where indicated in the cell below.

In [3]:
import json

#Set up access to Streaming Analytics service

def get_service_name():
    ## change the service name here, it would be Streaming Analytics in the screenshot above,
    # and "Streaming-Analytics" if you used the IoT starter kit
    service_name = "Streaming Analytics" 
    return service_name
def get_credentials():
    credentials = """enter_your_service_credentials_here"""
 
    return credentials


"""Submit the topology to the Streaming analytics service
"""
def submit_to_service(topo):
    service_name = get_service_name()
    credentials = get_credentials()
    vs={'streaming-analytics': [{'name': service_name, 'credentials': json.loads (credentials)}]}
    cfg = {}
    cfg[ConfigParams.VCAP_SERVICES] = vs
    cfg[ConfigParams.SERVICE_NAME] = service_name
    return submit('STREAMING_ANALYTICS_SERVICE', topo, cfg)


<a id="step22"></a>
### 2.2 Define helper functions


These are some utilities that will be used by your Streams application. 

The `get_event_data` function parses the data from an event you received. 

The `get_cmd` helper function will build the command tuple that is sent back to the Edgent application. The commands are JSON and have the following format:
```
{
 'deviceId': "id of device receiving the command",
  'typeId': "typeId of receiving device",
  'cmdId': "some id",
 'jsonString': {'d':{command payload as JSON object}}
}
```
You can get the device and type id by using the data from an event you received, the incoming tuple contains the event to which you are responding.

Run the following cell to define these functions.


In [4]:
import json
import math
from streamsx.topology.topology import Topology
from streamsx.topology.context import *
from streamsx.rest import StreamingAnalyticsConnection
from streamsx.topology import schema
import random


"""
This function will build the command tuple that is sent back
to the Edgent application. 
The commands are JSON, and the format is:

{
 'deviceId': "id of device receiving the command",
  'typeId': "typeId of receiving device",
  'cmdId': "some id",
 'jsonString': {'d':{command payload as JSON object}}
}
One way to get the device and type id is to use the data from an event we received
the incoming tuple contains the event we are responding to
"""
def get_cmd(tuple):
    #build the message you wish to send as a dictionary
    payload = {}

    payload["action"] = "Lights"
    payload["msg"] = "Message to Edgent from Streams\n:Alert code: 249"

    command_data =  {}
    command_data ["d"] = payload

    #convert the whole payload to json
    command_as_json = json.dumps(command_data)

    #build the command metadata. The device id and device type are on the tuple, but you could also specify them manually
    device_cmd ={}
    device_cmd["typeId"] = tuple["typeId"]
    device_cmd["cmdId"] = "display"
    device_cmd["deviceId"] = tuple["deviceId"]

    device_cmd["jsonString"] = command_as_json
    return device_cmd

"""Parse the data from an event we received"""
def get_event_data(tuple):
    payload_json = tuple["jsonString"]
    payload = json.loads(payload_json)
    return payload["d"]



<a id="step23"></a>
### 2.3 Create the Streams application

This application is going to process events from an IoT device running an Apache Edgent application. The application has a range sensor that determines the physical distance of an object from the sensor.  It will send events whenever an object is less than 4m away from the sensor. It sends the average, maximum, minimum, and standard deviation (STDDEV) of the distance of the object for the last 10 readings. The events are of the form:
```
{"name":"A","reading":{"N":2,"MIN":-2.384453313010427,"MAX":-2.0927794927623795,"MEAN":-2.2386164028864033,"STDDEV":0.20624453619198055}}
```
The Streams application will connect to the IoT platform to retrieve and print the events. Then it will also send a command back to the device to turn on the lights on the sensor. 

#### Create the Streams `Topology`

Streams applications are directed graphs with data moving from one operation to the next. A Streams application written in Python is called a `Topology`. The `Topology` we are creating will start by ingesting the events from the device and printing them:


In [5]:
from streamsx.topology.topology import Topology

from streamsx.rest import StreamsConnection
from streamsx.topology import schema
import json
 #define needed variables
COMMANDS_TOPIC = "streamsx/iot/device/commands/send" #topic to publish commands to
EVENTS_TOPIC = "streamsx/iot/device/events" #topic to subscribe to for events
incoming_schema =  schema.StreamSchema("tuple <rstring typeId, rstring deviceId, rstring eventId,rstring jsonString>")
cmd_schema = schema.StreamSchema('tuple<rstring typeId, rstring deviceId, rstring cmdId, rstring jsonString>')

#Topology object is the Streams application graph
topology = Topology('ReadingsFromIot')

#Subscribe to  events
events = topology.subscribe(EVENTS_TOPIC, incoming_schema,"AllEventsAsJSON")
sensor_events = events.filter(lambda tuple: tuple["eventId"] == "sensors","SensorEventsAsJSON")
readings = sensor_events.map(get_event_data,"ReadingsStream")
#print out readings
readings.print()


<a id="step24"></a>
### 2.4 Send a response for each event

Next, you will create a stream of commands to send back to the device. This is a very simple scenario, since you are not performing any major analysis but simply sending a response back for each event. Typically, you could do more complex analysis, or connect to a database, etc. 


In [6]:

#send a command
cmd_stream = sensor_events.map(get_cmd, "CommandsAsJSON")
#convert the commands stream to a SPL structured schema
commands_to_publish = cmd_stream.map(lambda x : (x["typeId"],x["deviceId"],x["cmdId"],x["jsonString"],), schema = cmd_schema, name="CommandsToPublish")

commands_to_publish.publish(COMMANDS_TOPIC, cmd_schema)



<a id="step25"></a>
### 2.5 Submit the `Topology` to the Streaming Analytics service

Finally, run the following cell to submit the Streams topology to your Streaming Analytics service:

In [ ]:
result = submit_to_service(topology)
print("Submitted job to the service, job id = " + str(result.job.id))


<a id="view"></a>
## 3. View the data

You can view both events received from the Edgent application to the console as well as the commands sent back to Edgent.

Since this application prints the received events from the Edgent application to the console, you can check the log viewer in the Streams Console to see the events received from the Edgent application. You can also view the output of the Edgent application to see that it does receive the commands from Streams. You can see messages like this:
```
Message to Edgent from Streams:
Alert code: 249
{"name":"B","reading":{"N":50,"MIN":-2.762152936700155,"MAX":5.7895310031906675,"MEAN":2.239131357345944,"STDDEV":2.199931362960884}}
Message to Edgent from Streams:
Alert code: 249
{"name":"B","reading":{"N":50,"MIN":-2.762152936700155,"MAX":5.7895310031906675,"MEAN":2.127842287474011,"STDDEV":2.2555865482121846}}
```


<a id="stop"></a>
## 4. Stop the application
From the Streams Console, cancel this application and the `IoTPlatform` job, otherwise, they can run indefinitely, consuming resources in your service. If you cancel the job, you must [re-run cell 2.5](#step25).

<a id="summary"></a>
## 5. Summary and next steps

Now that you have a basic Python application ingesting IoT data and running in the Streaming Analytics service, you can:

- Try the "Detect Malfunctioning Sensors" notebook:
    - <a href="https://www.youtube.com/watch?v=PDTSws6zULI" target="_blank" rel="noopener noreferrer">Watch the demo video on Youtube</a>, then 
    - <a href="https://dataplatform.ibm.com/exchange/public/entry/view/ec0aa15c6ab928b9b43ac0109d4395f1" target="_blank">Try the notebook out yourself</a>.
- Learn more about developing Streaming Analytics applications in Python: 
  - Take the <a href="https://developer.ibm.com/courses/all/streaming-analytics-basics-python-developers/" target="_blank">Streaming Analytics for Python developers course</a> on developerWorks.
  - <a href="http://ibmstreams.github.io/streamsx.documentation/docs/python/1.6/python-appapi-devguide/" target="_blank">Consult the Streams Python API Developer Guide</a>.
- For more Edgent and IoTP documentation, you can check out the <a href="http://edgent.incubator.apache.org/docs/quickstart.html" target="_blank" rel="noopener noreferrer">Edgent to Quickstart guide</a>.
- Visit <a href="https://developer.ibm.com/streamsdev/" target="_blank">Streamsdev, the Streams developer community</a>, for more useful resources about Streams.

Happy Streaming!

 

<a id="authors"></a> 
### Author

**Natasha D'Silva** is a software developer at IBM Canada who specializes in streaming technology and cloud solutions.

<hr>
Copyright &copy; IBM Corp. 2018. This notebook and its source code are released under the terms of the Apache 2.0 License.